# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [106]:
# Details Student 1:
ID = 207058132
Name = "Shaked Benno"
# Details Student 2:
ID = 207191644
Name = "May Yashar"

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [107]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [108]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [109]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [110]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [111]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [112]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [113]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [114]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [115]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [116]:
# YOUR CODE HERE

# Split the columns'story' from 'gender' in Dataframe in order to procceed to clean the data and work on it with the models
X = df_train["story"]
Y = df_train["gender"]

X.head(3)
Y.head(3)

0    כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...
1    לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...
2    מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...
Name: story, dtype: object

0    m
1    m
2    f
Name: gender, dtype: object

In [117]:
# YOUR CODE HERE

# Part 1 Cleaning  - All stopwords and symbols are removed + removed empty spaces
def clean_words(data):
    # 1St pass Cleaning - Reconstruction but now with more features
    string_test = data.str.replace('\W', ' ').str.split(" ")
    new_string_test = []
    for words in string_test:
        new_string_test.append([word for word in words if word])

    # Part 2 Cleaning - if there is a single word with a count of 1 , it should be added to the word behind it.
    string_test = []
    old = ""
    for line in new_string_test:
        line_test = []
        for word in line:
            if len(word) == 1 and not word.isnumeric() and word.isalnum():
                _ = line_test.pop()
                line_test.append(old + word)
            else:
                line_test.append(word)
            old = word
        string_test.append(line_test)
    return string_test

In [118]:
# YOUR CODE HERE

def numerize_words(string_test, file_name):
    # Transform the data back into Pandas Dataframe with more features, so we run the models with it

    # This process takes time, It takes some time to count all words for every row
    # This might take more than 10 minutes
    data_frame = pd.DataFrame()
    i = 0
    all_rows = str(len(string_test))
    for row in string_test:
        i += 1
        new_row = pd.DataFrame()
        for word in row:
            if word not in new_row.columns:
                new_row.loc[0, word] = 1
            else:
                new_row.loc[0, word] = new_row.loc[0, word] + 1
        data_frame = pd.concat([data_frame, new_row], ignore_index=True)
        # Show how much progress we have, It takes some time to count all words
        print(str(i) + " / " + str(all_rows), end='\r')
    data_frame.head(1)
    # Save progress so after the first time it should be fast to resume position
    data_frame.to_csv(file_name, index=False)
    print("Checkpoint numerized words - Saved")
    return data_frame

In [119]:
# Add as many code cells as you need
# YOUR CODE HERE

# 2ND pass cleaning
def clean_dataframe(data_frame, file_name):
    # Fill Nan with 0
    data_frame = data_frame.replace(np.nan, 0)
    print(data_frame.head(3))
    data_frame.to_csv(file_name, index=False)
    print("Checkpoint numerized words counts fixed- Work Saved")
    return data_frame


In [120]:
# Create Guass model classifier
def gaussian(XTrain, XTest, yTrain, yTest, version):
    if version == 2:
        print("------ Multinomial Gaussian ------")
        gnb = MultinomialNB()
    else:
        print("------ Gaussian ------")
        gnb = GaussianNB()

    # Train classifier
    gnb.fit(XTrain, yTrain)

    # Use the trained classifier to predict
    y_predict = gnb.predict(XTest)
    y_pred_train = gnb.predict(XTrain)

    # Evaluate Accuracy
    print('Accuracy on Train data=', metrics.accuracy_score(y_true=yTrain, y_pred=y_pred_train))
    print('Accuracy on test data=', metrics.accuracy_score(y_true=yTest, y_pred=y_predict))
    return gnb

In [121]:
# Create Decision Tree model Classifier 
def decision_tree(XTrain, XTest, yTrain, yTest):
    print("------ Decision Tree ------")
    decisionTree = tree.DecisionTreeClassifier()

    # fit the tree with the training data
    decisionTree = decisionTree.fit(XTrain, yTrain)

    # predict with the training data
    y_pred_train = decisionTree.predict(XTrain)
    
    # measure accuracy
    print('Accuracy on training data =', metrics.accuracy_score(y_true=yTrain, y_pred=y_pred_train))

    # predict with the test data
    y_predict = decisionTree.predict(XTest)
    
    # measure accuracy
    print('Accuracy on test data =', metrics.accuracy_score(y_true=yTest, y_pred=y_predict))
    return decisionTree

In [126]:
# Create PCA model classifier
def pca(XTrain, XTest, yTrain, yTest):
    print("------ PCA ------")
    sc = StandardScaler()

    X_train = sc.fit_transform(XTrain)
    X_test = sc.transform(XTest)
    pca = PCA(n_components=2)

    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    explained_variance = pca.explained_variance_ratio_
    classifier = DecisionTreeClassifier()
    classifier.fit(X_train, yTrain)
    y_pred_train = classifier.predict(X_train)
    y_predict = classifier.predict(X_test)
    print('Accuracy on Train data=', metrics.accuracy_score(y_true=yTrain, y_pred=y_pred_train))
    print('Accuracy on test data=', metrics.accuracy_score(y_true=yTest, y_pred=y_predict))
    return pca

In [123]:
# Set names for files to load up work faster after first iteration

new_df = dataframe = test_new_df = test_dataframe = 0
num_words = "numerize_words.csv"
words_clean = "numerize_words_cleaned.csv"
test_num_words = "test_numerize_words.csv"
test_words_clean = "test_numerize_words_cleaned.csv"

In [124]:
# Resume from X position if stopped at the middle sometime

try:
    dataframe = pd.read_csv(num_words)
    print("Resuming part 1")
    new_df = pd.read_csv(words_clean)
    print("Resuming part 2")

    test_dataframe = pd.read_csv(test_num_words)
    print("Resuming part 3")
    test_new_df = pd.read_csv(test_words_clean)
    print("Resuming part 4")
except:
    if type(dataframe) is not pd.DataFrame:
        dataframe = None

    if type(new_df) is not pd.DataFrame:
        new_df = None
        
    if type(test_dataframe) is not pd.DataFrame:
        test_dataframe = None

    if type(test_new_df) is not pd.DataFrame:
        test_new_df = None

In [125]:
# Clean story from dataframe in both the data we train the model and the test to predict

# Here we work on the actual training data
if type(new_df) is not pd.DataFrame:
    if type(dataframe) is not pd.DataFrame:
        print("Cleaning sentences")
        dataframe = clean_words(X)
        print("Counting word appearance")
        dataframe = numerize_words(dataframe, num_words)
    print("Cleaning Nan from Dataframe")
    dataframe = clean_dataframe(dataframe, words_clean)
else:
    dataframe = new_df
    new_df = None

# Here we work at the test data
# Same thing like the one above but on the training data.
# We know its doubled code, but in order to ensure progress save ,we have to do it.
if type(test_new_df) is not pd.DataFrame:
    if type(test_dataframe) is not pd.DataFrame:
        print("Cleaning test sentences")
        test_dataframe = clean_words(df_test.story)
        print("Counting test word appearance")
        test_dataframe = numerize_words(test_dataframe, test_num_words)
    print("Cleaning Nan from test Dataframe")
    test_dataframe = clean_dataframe(test_dataframe, test_words_clean)
else:
    test_dataframe = test_new_df
    test_new_df = None


Cleaning sentences
Counting word appearance
Checkpoint numerized words - Saved
Cleaning Nan from Dataframe
   כשחבר  הזמין  אותי  לחול    לא  באמת  חשבתי   שזה  יקרה  פשוט  אמרתי    לו  \
0  1.000  1.000 1.000 1.000 6.000 1.000  1.000 5.000 1.000 1.000  2.000 1.000   
1  0.000  0.000 0.000 0.000 6.000 0.000  0.000 1.000 0.000 0.000  1.000 2.000   
2  0.000  0.000 0.000 0.000 7.000 0.000  0.000 0.000 0.000 0.000  0.000 0.000   

     כן  ותיארתי  לעצני  יתבטל  אחרי  שבועיים  בערך   אני  מקבל  טלפוןם  ממנו  \
0 1.000    1.000  1.000  1.000 1.000    1.000 2.000 4.000 1.000   1.000 2.000   
1 0.000    0.000  0.000  0.000 1.000    0.000 0.000 2.000 0.000   0.000 0.000   
2 0.000    0.000  0.000  0.000 3.000    0.000 0.000 3.000 0.000   0.000 0.000   

   שומע  מצאתי  אחלה  מקודות  שנוטכל  טייל   בהם   ואז  הבנתי  הולך  לקרות  \
0 1.000  1.000 1.000   1.000   1.000 1.000 1.000 2.000  2.000 1.000  1.000   
1 0.000  0.000 0.000   0.000   0.000 0.000 0.000 0.000  0.000 0.000  0.000   
2 0.000  

Checkpoint numerized words counts fixed- Work Saved
Cleaning test sentences
Counting test word appearance
Checkpoint numerized words - Saved
Cleaning Nan from test Dataframe
     כל   קיץ   אני  והמשפחה  נוסעים  לארצות  הברית   לוס   אנג    לס  \
0 7.000 1.000 3.000    1.000   1.000   1.000  1.000 1.000 1.000 1.000   
1 1.000 0.000 3.000    0.000   0.000   0.000  0.000 0.000 0.000 0.000   
2 4.000 0.000 3.000    0.000   0.000   0.000  0.000 0.000 0.000 0.000   

   קליפורניה  בכדי  לבקר    את  המשפחה    של   אמא   שלי  שגרה    שם    זה  \
0      1.000 1.000 1.000 2.000   1.000 3.000 1.000 3.000 1.000 4.000 3.000   
1      0.000 0.000 0.000 6.000   0.000 9.000 0.000 2.000 0.000 3.000 8.000   
2      0.000 0.000 0.000 6.000   0.000 2.000 0.000 3.000 0.000 0.000 4.000   

   תמיד  עושה   טוב  ללכת  לראות  משפחה  שהתגעגענו  אליהם    כך  ושלא  ראינו  \
0 1.000 1.000 1.000 3.000  1.000  1.000      1.000  1.000 1.000 1.000  1.000   
1 2.000 0.000 1.000 0.000  0.000  0.000      0.000  0.000 1.

Checkpoint numerized words counts fixed- Work Saved


In [127]:

def align_features_numbers_predict(train, to_predict):
    for column in train.columns:
        if column not in to_predict.columns:
            to_predict[column] = 0
    return to_predict


def align_features_numbers_train(train, to_predict):
    for column in to_predict.columns:
        if column not in train.columns:
            train[column] = 0
    return train

# Align the numbers of features from the train data and test data
test_dataframe = align_features_numbers_predict(dataframe, test_dataframe)
X = align_features_numbers_train(dataframe, test_dataframe)
print("Alignment has finished!")


Alignment has finished!


In [128]:
# Setting the rest of the parameters to work on models
Y = Y.replace('m', 0).replace('f', 1).astype(int)
rand_state = 42
test_size = 0.1

# Splitting the data for the models to run
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, random_state=rand_state, test_size=test_size)

# Call the models for train and calculate score

clf_gaussian = gaussian(XTrain, XTest, yTrain, yTest, 1)
clf_multy_gaussian = gaussian(XTrain, XTest, yTrain, yTest, 2)
clf_decision_tree = decision_tree(XTrain, XTest, yTrain, yTest)
clf_pca = pca(XTrain, XTest, yTrain, yTest)


------ Gaussian ------
Accuracy on Train data= 1.0
Accuracy on test data= 0.75
------ Multinomial Gaussian ------
Accuracy on Train data= 0.9675036927621861
Accuracy on test data= 0.75
------ Decision Tree ------
Accuracy on training data = 1.0
Accuracy on test data = 0.6710526315789473
------ PCA ------
Accuracy on Train data= 1.0
Accuracy on test data= 0.6052631578947368


In [129]:
# Our choice for model to predict

pred = clf_multy_gaussian.predict(test_dataframe)
df_predicted = pd.DataFrame({'test_story_id': test_dataframe.index, 'predicted': pred})
df_predicted.predicted = df_predicted.predicted.replace(1, 'm').replace(0, 'f')

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [130]:
df_predicted.to_csv('classification_results.csv',index=False)